# Perfilamiento del Código

## Librerías, funciones y datos para las pruebas

In [2]:
%pip install -q --user line_profiler
%pip install -q --user memory_profiler
%pip install -q --user psutil
%pip install -q --user guppy3
#%pip install -q --user pkgcore

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
import numpy as np
import pandas as pd
import os
import time
import warnings; warnings.simplefilter('ignore')
import platform
from datetime import datetime
import psutil

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
%load_ext memory_profiler
%load_ext line_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [6]:
import math
from guppy import hpy
from scipy.integrate import quad

In [7]:
# Importamos el módulo con nuestras funciones
import sys
sys.path.append('./../../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#Own Library modules
import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.pca import todoJunto

In [8]:
df = pd.read_csv('../../data/nndb_flat.csv', encoding = "UTF-8")

In [9]:
df.drop(df.columns[df.columns.str.contains('_USRDA')].values, 
        inplace=True, axis=1)
df = df.drop(columns=['ID','FoodGroup','ShortDescrip','Descrip','CommonName','MfgName','ScientificName'])

In [10]:
####Se estandarizan los datos.
scaler = preprocessing.StandardScaler()
df_nutrientes = scaler.fit_transform(df)

## Equipo Utilizado

In [102]:
#Función para escalar cifras en Bytes a las unidades correspondientes
def get_size(bytes, suffix="B"):

    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*40, "Información del Sistema", "="*40)
uname = platform.uname()
print(f"Sistema: {uname.system}")
print(f"Nombre: {uname.node}")
print(f"OS: {uname.release}")
print(f"Versión: {uname.version}")
print(f"Máquina: {uname.machine}")
print(f"Procesador: {uname.processor}")

# Detalles de la memoria
print("="*40, "Memoria", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Desponible: {get_size(svmem.available)}")
print(f"Utilizada: {get_size(svmem.used)}")
print(f"Porcentaje: {svmem.percent}%")
print("="*40, "CPU", "="*40)
# number of cores
print("Cores Físicos:", psutil.cpu_count(logical=False))
print("Cores Totales:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Frecuencia del CPU : {psutil.cpu_freq()}%")
# Disk Information
print("="*40, "Discos Duros", "="*40)
print("Particiones:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Dispositivo: {partition.device} ===")
    print(f"  Montura: {partition.mountpoint}")
    print(f"  Tipo de Sistema de Archivos: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Tamaño Total: {get_size(partition_usage.total)}")
    print(f"  Usado: {get_size(partition_usage.used)}")
    print(f"  Libre: {get_size(partition_usage.free)}")
    print(f"  Porcentaje: {partition_usage.percent}%")

======================================== Información del Sistema ========================================
Sistema: Windows
Nombre: DESKTOP-F8PSOT5
OS: 10
Versión: 10.0.17763
Máquina: AMD64
Procesador: Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
======================================== Memoria ========================================
Total: 15.88GB
Desponible: 6.28GB
Utilizada: 9.59GB
Porcentaje: 60.4%
======================================== CPU ========================================
Cores Físicos: 4
Cores Totales: 4
Frecuencia del CPU : scpufreq(current=2501.0, min=0.0, max=2501.0)%
======================================== Discos Duros ========================================
Particiones:
=== Dispositivo: C:\ ===
  Montura: C:\
  Tipo de Sistema de Archivos: NTFS
  Tamaño Total: 1.68TB
  Usado: 1.19TB
  Libre: 502.10GB
  Porcentaje: 70.8%
=== Dispositivo: D:\ ===
  Montura: D:\
  Tipo de Sistema de Archivos: NTFS
  Tamaño Total: 25.00GB
  Usado: 2.16GB
  Libre: 22.84GB
  Por

# 1. Pruebas de Tiempo

El objetivo de este notebook es perfilar y los resultados de PCA de cada una de nuestras funciones:
+ PCA usando la función SVD de numpy.
+ PCA a partir del algoritmo QR
+ PCA a partir del método de la potencia

## 1.1 A través de time

In [123]:
start_time1 = time.time()
todoJunto.PCA_from_sklearn(df_nutrientes)
end_time1 = time.time()

start_time2 = time.time()
todoJunto.PCA_from_SVD(df_nutrientes)
end_time2 = time.time()

start_time3 = time.time()
todoJunto.PCA_from_QR_vf(df_nutrientes)
end_time3 = time.time()

start_time4 = time.time()
todoJunto.PCA_from_potencia(df_nutrientes)
end_time4 = time.time()

In [64]:
secs1 = end_time1-start_time1
secs2 = end_time2-start_time2
secs3 = end_time3-start_time3
secs4 = end_time4-start_time4
print("PCA_from_sklearn tomó",secs1,"segundos" )
print("PCA_from_SVD tomó",secs2,"segundos" )
print("PCA_from_QR tomó",secs3,"segundos" )
print("PCA_from_potencia tomó",secs4,"segundos" )

PCA_from_sklearn tomó 0.015001058578491211 segundos
PCA_from_SVD tomó 0.006003618240356445 segundos
PCA_from_QR tomó 1.2579970359802246 segundos
PCA_from_potencia tomó 0.4570002555847168 segundos


Las funciones provenientes de sklearn y numpy fueron claramente más rápidas que las que elaboramos para el proyecto.

## 1.2 A través de %timeit

Al igual que en las notas, se utilizará un loop de tamaño $5$ y $20$ repeticiones.

In [29]:
%timeit -n 5 -r 20 todoJunto.PCA_from_sklearn(df_nutrientes)

15.4 ms ± 506 µs per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

In [32]:
%timeit -n 5 -r 20 todoJunto.PCA_from_SVD(df_nutrientes)

6.42 ms ± 483 µs per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

In [60]:
%timeit -n 5 -r 20 todoJunto.PCA_from_QR_vf(df_nutrientes)

1.24 s ± 69.8 ms per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

In [62]:
%timeit -n 5 -r 20 todoJunto.PCA_from_potencia(df_nutrientes)

435 ms ± 9.12 ms per loop (mean ± std. dev. of 20 runs, 5 loops each)


No se observan variaciones notorias al correrlo con 20 repeticiones.

Al igual que con time, las funciones provenientes de sklearn y numpy fueron claramente más rápidas que las que elaboramos para el proyecto.

# 2 Pruebas de uso de CPU

## 2.1 Con %prun Y %lprun

Se perfila primero con *%prun* a cada función  aquellas funciones que gastan un mayor tiempo de ejecución y posteriormente perfilarlas con line_profiler.

### PCA_from_sklearn

In [71]:
%prun -s cumulative todoJunto.PCA_from_sklearn(df_nutrientes)

         2750 function calls (2677 primitive calls) in 0.022 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.022    0.022 {built-in method builtins.exec}
        1    0.000    0.000    0.022    0.022 <string>:1(<module>)
        1    0.000    0.000    0.021    0.021 todoJunto.py:20(PCA_from_sklearn)
        2    0.001    0.000    0.021    0.010 _pca.py:347(fit_transform)
        2    0.000    0.000    0.020    0.010 _pca.py:381(_fit)
        2    0.000    0.000    0.015    0.008 _pca.py:423(_fit_full)
        2    0.011    0.005    0.011    0.006 decomp_svd.py:16(svd)
        2    0.000    0.000    0.005    0.003 validation.py:350(check_array)
        2    0.003    0.001    0.003    0.002 extmath.py:496(svd_flip)
    34/32    0.002    0.000    0.002    0.000 {built-in method numpy.array}
       10    0.000    0.000    0.002    0.000 {built-in method numpy.core._multiarray_umath.implement_a

In [72]:
%lprun -f todoJunto.PCA_from_sklearn todoJunto.PCA_from_sklearn(df_nutrientes)

Timer unit: 1e-07 s

Total time: 0.0289618 s
File: ./../..\src\pca\todoJunto.py
Function: PCA_from_sklearn at line 20

Line #      Hits         Time  Per Hit   % Time  Line Contents
    20                                           def PCA_from_sklearn(X):
    21                                               """
    22                                               componentes_principales(X): Función que devuelve las componentes principales.
    23                                               
    24                                               Parámetros
    25                                               ----------
    26                                               n_components: número de componentes. 
    27                                               svd_solver: str {‘auto’, ‘full’, ‘arpack’, ‘randomized’}
    28                                                   Se elige 'full', lo que significa que se ejecuta completamente SVD llamando al 
    29                              

Es notorio que las transformaciones que ejecuta PCA al llamar a decomp_svd.py y ejecutar las transformaciones de la función fit_transform() son los elementos que consumen más tiempo. Al tratarse de una función propia de sklearn, no existe demasiado margen para recortar tiempos.

### PCA_from_SVD

In [73]:
%prun -s cumulative todoJunto.PCA_from_SVD(df_nutrientes)

         60 function calls in 0.007 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.007    0.007 {built-in method builtins.exec}
        1    0.001    0.001    0.007    0.007 <string>:1(<module>)
        1    0.001    0.001    0.006    0.006 todoJunto.py:63(PCA_from_SVD)
        3    0.000    0.000    0.003    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.003    0.003 <__array_function__ internals>:2(svd)
        1    0.003    0.003    0.003    0.003 linalg.py:1458(svd)
        3    0.001    0.000    0.001    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.000    0.000 {method 'mean' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 _methods.py:134(_mean)
        3    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000   

In [74]:
%lprun -f todoJunto.PCA_from_SVD todoJunto.PCA_from_SVD(df_nutrientes)

Timer unit: 1e-07 s

Total time: 0.0119768 s
File: ./../..\src\pca\todoJunto.py
Function: PCA_from_SVD at line 63

Line #      Hits         Time  Per Hit   % Time  Line Contents
    63                                           def PCA_from_SVD(A):
    64                                               """
    65                                               Función para PCA a partir de la SVD de numpy 
    66                                               params: A			matriz de datos
    67                                                       num_componentes 	número de componentes deseados
    68                                           
    69                                               return: valores_singulares	Los valores singulares de la descomposición SVD
    70                                           	    componentes		Los coeficientes para calcular los componentes principales
    71                                           	    Z			Los datos transformados (componentes princip

El llamado a linalg.svd es el elemento que consume casi la mitad del tiempo. Al tratarse de una función propia de numpy, no existe demasiado margen para recortar tiempos.

### PCA_from_QR_vf

In [75]:
%prun -s cumulative todoJunto.PCA_from_QR_vf(df_nutrientes)

         769281 function calls (769280 primitive calls) in 1.527 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.527    1.527 {built-in method builtins.exec}
        1    0.002    0.002    1.527    1.527 <string>:1(<module>)
        1    0.003    0.003    1.524    1.524 todoJunto.py:147(PCA_from_QR_vf)
        1    0.016    0.016    1.518    1.518 algoritmo_QR.py:10(eigenvectores_eigenvalores_QR_vf)
      451    0.215    0.000    1.488    0.003 funciones_examen_QR.py:168(matriz_Q_R)
    20746    0.267    0.000    0.736    0.000 funciones_examen_QR.py:208(Q_j)
      451    0.156    0.000    0.531    0.001 funciones_examen_QR.py:112(matriz_auxiliar_Arv)
83889/83888    0.133    0.000    0.424    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    31119    0.023    0.000    0.292    0.000 <__array_function__ internals>:2(outer)
    10373    0.138    0.000    0.249

In [85]:
%lprun -f todoJunto.PCA_from_QR_vf todoJunto.PCA_from_QR_vf(df_nutrientes)

Timer unit: 1e-07 s

Total time: 1.6004 s
File: ./../..\src\pca\todoJunto.py
Function: PCA_from_QR_vf at line 147

Line #      Hits         Time  Per Hit   % Time  Line Contents
   147                                           def PCA_from_QR_vf(data,niter = 450):
   148                                               """
   149                                               Función para PCA a partir de los eigenvectores  
   150                                               params: data:			matriz de datos
   151                                                       niter:      número de iteraciones máximas    
   152                                               
   153                                               
   154                                               return:     componentes		Los coeficientes para calcular los componentes principales (eigenvectores de la matriz de covarianzas)
   155                                                           Z			Los datos transformados (c

La mayor parte del tiempo lo realiza la función eigenvectores_eigenvalores_QR_vf la cual tuvo como elementos con más utilización de tiempo a los llamados a las siguientes funciones
* **matriz_Q_R**
* **matriz_auxiliar_Arv**

por lo que se realiza un lprun de las mismas. Se comprueba que el cómputo más intensivo se encuentra en las iteraciones ocurridas en estas funciones, en específico en la multiplicación que genera las componentes r de la matriz R dentro del algoritmo matriz_auxiliar_Arv

In [92]:
    # convertir a array
    A = np.array(df_nutrientes)
    
    # Centrar los datos
    mean_vec = np.mean(A, axis=0)
    datos_centrados = (A - mean_vec)

    # Matriz de Covarianzas
    C = (A - mean_vec).T.dot((A - mean_vec)) / (A.shape[0]-1)
    
    n, p = C.shape
    columnas = n - p
    ceros = np.zeros((n,columnas))
    
    # Matriz inicial
    A0 = np.append(C, ceros, axis = 1)


from src.pca.funciones_examen_QR import matriz_Q_R
%lprun -f matriz_Q_R matriz_Q_R(A0)

Timer unit: 1e-07 s

Total time: 0.0045808 s
File: ./../..\src\pca\funciones_examen_QR.py
Function: matriz_Q_R at line 168

Line #      Hits         Time  Per Hit   % Time  Line Contents
   168                                           def matriz_Q_R(A,ind_singular=False):
   169                                               """
   170                                               Función que devuelve la matriz R y Q de la factorización QR de una matriz A
   171                                               
   172                                               params: A    Matriz (mxn)
   173                                               return: Q    Matriz Q (mxm) de la factorización A=QR
   174                                                       R    Matriz Q (mxm) de la factorización A=QR
   175                                               """
   176                                               
   177                                               #Se checa que los parámetros se

In [95]:
    # convertir a array
    A = np.array(df_nutrientes)
    
    # Centrar los datos
    mean_vec = np.mean(A, axis=0)
    datos_centrados = (A - mean_vec)

    # Matriz de Covarianzas
    C = (A - mean_vec).T.dot((A - mean_vec)) / (A.shape[0]-1)
    
    n, p = C.shape
    columnas = n - p
    ceros = np.zeros((n,columnas))
    
    # Matriz inicial
    A0 = np.append(C, ceros, axis = 1)


from src.pca.funciones_examen_QR import matriz_auxiliar_Arv
%lprun -f matriz_auxiliar_Arv matriz_auxiliar_Arv(A0,ind_singular=False)

Timer unit: 1e-07 s

Total time: 0.0013027 s
File: ./../..\src\pca\funciones_examen_QR.py
Function: matriz_auxiliar_Arv at line 112

Line #      Hits         Time  Per Hit   % Time  Line Contents
   112                                           def matriz_auxiliar_Arv(A,ind_singular=False):
   113                                               """
   114                                               Función que genera una matriz que contiene los elementos r distintos de cero de la 
   115                                               matriz R y las entradas de los vectores householder v (excepto la primera), con los 
   116                                               cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
   117                                               
   118                                               params: A      Matriz (mxn) de la que se desea obtner factorización QR
   119                                                   

Por la naturaleza iterativa del algoritmo así como del tamaño de la matriz introducida, se justifica que el algoritmo es computacionalmente más intensivo que los demás que se encuentran mucho más optimizados dentro de las librerías Numpy y ScyKit Learn de Python. **Se sugiere paralelizar esta parte del código o la función en su totalidad para obtener mejores resultados.**

### PCA_from_potencia

In [104]:
%prun -s cumulative todoJunto.PCA_from_potencia(df_nutrientes)

         736545 function calls (690545 primitive calls) in 0.585 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.585    0.585 {built-in method builtins.exec}
        1    0.000    0.000    0.585    0.585 <string>:1(<module>)
        1    0.002    0.002    0.585    0.585 todoJunto.py:196(PCA_from_potencia)
        1    0.001    0.001    0.580    0.580 metodo_potencia_deflation.py:38(power_deflation)
       23    0.125    0.005    0.579    0.025 metodo_potencia_deflation.py:5(power_iteration)
138049/92049    0.132    0.000    0.394    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    46000    0.026    0.000    0.367    0.000 <__array_function__ internals>:2(norm)
    46000    0.161    0.000    0.310    0.000 linalg.py:2316(norm)
    92001    0.050    0.000    0.167    0.000 <__array_function__ internals>:2(dot)
    46092    0.024    0.000    0.024    0.000 {me

In [105]:
%lprun -f todoJunto.PCA_from_potencia todoJunto.PCA_from_potencia(df_nutrientes)

Timer unit: 1e-07 s

Total time: 0.838369 s
File: ./../..\src\pca\todoJunto.py
Function: PCA_from_potencia at line 196

Line #      Hits         Time  Per Hit   % Time  Line Contents
   196                                           def PCA_from_potencia(X):
   197                                               """
   198                                               Función que calcula PCA a partir del método de la potencia y deflation de Hotteling  
   199                                               params: A:			matriz de datos
   200                                               
   201                                               
   202                                               return:     eigenvalues		Numpy array con los eigenvectores de A
   203                                                           eigenvectors	Numpy array con los correspondientes eigenvectores de A 
   204                                               
   205                                            

Claramente la parte más intensiva en tiempo y cómputo corresponde a la **iteración** que realiza el **método de deflation:** el cual tiene el número de iteraciones en hard-code con 2000 de las mismas.

In [119]:
#Calculamos la matriz de covarianzas
cov = np.dot(df_nutrientes.T, df_nutrientes)/df_nutrientes.shape[0]

from src.pca.metodo_potencia_deflation import power_deflation
%lprun -f power_deflation power_deflation(cov,2000)

Timer unit: 1e-07 s

Total time: 0.861516 s
File: ./../..\src\pca\metodo_potencia_deflation.py
Function: power_deflation at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                           def power_deflation(A,iter):
    39                                               """
    40                                               Función que itera el método de la potencia, mediante deflation de Hotteling  
    41                                               params: A:			matriz (en nuestro caso, la de covarianzas)
    42                                                       iter:      número de iteraciones a realizar    
    43                                               
    44                                               
    45                                               return:     eigenvalues		Numpy array con los eigenvectores de A
    46                                                           eigenvectors	Numpy array con 

In [121]:
cov = np.dot(df_nutrientes.T, df_nutrientes)/df_nutrientes.shape[0]
A_def = cov.copy()
from src.pca.metodo_potencia_deflation import power_iteration
%lprun -f power_iteration power_iteration(A_def,2000)

Timer unit: 1e-07 s

Total time: 0.0503572 s
File: ./../..\src\pca\metodo_potencia_deflation.py
Function: power_iteration at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def power_iteration(A, num_simulations: int):
     6                                               """
     7                                               Función que implementa el método de la potencia  
     8                                               params: A:			matriz (en nuestro caso, la de covarianzas)
     9                                                       num_simulations:      número de iteraciones a realizar    
    10                                               
    11                                               
    12                                               return:     m_k		Eigenvector dominante
    13                                                           b_k		Eigenvector corresondiente
    14                

Se observa que la normalización del Eigenvector correspondiente y sus respectivas iteraciones son las más intensivas en consumo de tiempo por lo que se sugiere **paralelizar** esa parte del código o bien la función en su totalidad

## 3. Uso de memoria RAM

### 3.1 A través de %memit

In [11]:
%memit -c todoJunto.PCA_from_sklearn(df_nutrientes)

peak memory: 166.97 MiB, increment: 48.18 MiB


In [12]:
%memit -c todoJunto.PCA_from_SVD(df_nutrientes)

peak memory: 167.29 MiB, increment: 46.22 MiB


In [13]:
%memit -c todoJunto.PCA_from_QR_vf(df_nutrientes)

peak memory: 164.20 MiB, increment: 43.05 MiB


In [14]:
%memit -c todoJunto.PCA_from_potencia(df_nutrientes)

peak memory: 164.31 MiB, increment: 43.03 MiB


Los resultados de Memit reflejan una utilización similar de la memoria en todas las funciones, siendo las que implementamos en el proyectos ligeramente mejores **(aproximadamente entre un 1%y 2% mejores)**.

### 3.2 A través de heapy

In [22]:
hp = hpy()

In [27]:
hp.setrelheap()
h = hp.heap()
todoJunto.PCA_from_sklearn(df_nutrientes)
h = hp.heap()
h

Partition of a set of 15 objects. Total size = 2193 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      3  20     1376  63      1376  63 types.FrameType
     1      3  20      200   9      1576  72 list
     2      1   7      144   7      1720  78 types.CodeType
     3      2  13      128   6      1848  84 tuple
     4      1   7      112   5      1960  89 dict of _ast.Module
     5      1   7       64   3      2024  92 types.MethodType
     6      1   7       56   3      2080  95 _ast.Module
     7      1   7       45   2      2125  97 bytes
     8      1   7       40   2      2165  99 _thread.lock
     9      1   7       28   1      2193 100 int

In [28]:
hp2 = hpy()
hp2.setrelheap()
h2 = hp2.heap()
todoJunto.PCA_from_SVD(df_nutrientes)
h2 = hp2.heap()
h2

Partition of a set of 18 objects. Total size = 2337 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      3  17     1376  59      1376  59 types.FrameType
     1      3  17      200   9      1576  67 list
     2      1   6      144   6      1720  74 types.CodeType
     3      2  11      128   5      1848  79 tuple
     4      1   6      112   5      1960  84 dict of _ast.Module
     5      3  17       92   4      2052  88 int
     6      1   6       80   3      2132  91 builtins.weakref
     7      1   6       64   3      2196  94 types.MethodType
     8      1   6       56   2      2252  96 _ast.Module
     9      1   6       45   2      2297  98 bytes
    10      1   6       40   2      2337 100 _thread.lock

In [29]:
hp3 = hpy()
hp3.setrelheap()
h3 = hp3.heap()
todoJunto.PCA_from_QR_vf(df_nutrientes)
h3 = hp3.heap()
h3

Partition of a set of 18 objects. Total size = 2337 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      3  17     1376  59      1376  59 types.FrameType
     1      3  17      200   9      1576  67 list
     2      1   6      144   6      1720  74 types.CodeType
     3      2  11      128   5      1848  79 tuple
     4      1   6      112   5      1960  84 dict of _ast.Module
     5      3  17       92   4      2052  88 int
     6      1   6       80   3      2132  91 builtins.weakref
     7      1   6       64   3      2196  94 types.MethodType
     8      1   6       56   2      2252  96 _ast.Module
     9      1   6       45   2      2297  98 bytes
    10      1   6       40   2      2337 100 _thread.lock

In [30]:
hp4 = hpy()
hp4.setrelheap()
h4 = hp.heap()
todoJunto.PCA_from_potencia(df_nutrientes)
h4 = hp.heap()
h4

Partition of a set of 303 objects. Total size = 39529 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0     22   7    11368  29     11368  29 types.FrameType
     1     37  12     5279  13     16647  42 str
     2     16   5     4224  11     20871  53 dict (no owner)
     3     30  10     2424   6     23295  59 list
     4     30  10     2000   5     25295  64 tuple
     5      7   2     1288   3     26583  67 zmq.sugar.frame.Frame
     6     10   3     1120   3     27703  70 dict of _ast.Name
     7      8   3     1088   3     28791  73 function
     8     10   3     1078   3     29869  76 bytes
     9     17   6      816   2     30685  78 builtins.cell
<31 more rows. Type e.g. '_.more' to view.>

Con heapy podemos darnos cuenta que heapy muestra resultados distintos a memit ya que es posible que exista un memory leak en la función del método de la potencia pues genera objetos de tipo frame que demandan más memoria, provenientes de las iteraciones que identificamos previamente en los ejercicios con el CPU.